# Chainerの簡単なサンプル

Chainerのドキュメント（イントロダクション）を実行してみる

http://docs.chainer.org/en/stable/tutorial/basic.html


In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

* 順伝播はネットワークの定義になる
* 順伝播計算をはじめるために、入力配列をVariableオブジェクトに変換する
* １次元のndarrayでやってみる

In [2]:
x_data = np.array([5], dtype=np.float32)

In [3]:
x = Variable(x_data)

* 次の式を実行してみる
\begin{equation*} 
y = x^2 - 2x + 1
\end{equation*}

In [4]:
y = x**2 - 2 * x + 1

* yもVariableオブジェクトのインスタンスになる
* Variableオブジェクトのdata属性にアクセスして値を取得できる

In [5]:
y.data

array([ 16.], dtype=float32)

* Variable型は計算結果を保持するだけでなく、計算グラフも保持する
* backwardメソッドを呼び出すことで、微分の計算も可能

In [6]:
y.backward()

* これで誤差逆伝播が実行される
* 計算された勾配は入力の変数xのgrad属性に保存

In [7]:
x.grad

array([ 8.], dtype=float32)

* 中間変数の勾配は、通常メモリ効率のために解放されている
* その勾配の情報を保存するために、retain_grad引数をbackwardメソッドにTrueで渡します

In [8]:
z = 2*x
y = x**2 - z + 1
y.backward(retain_grad=True)

In [9]:
z.grad

array([-1.], dtype=float32)

In [10]:
x.grad

array([ 16.], dtype=float32)

* これらすべての計算は多要素配列に一般化できます
* もし多要素配列の変数をすべて勾配計算したい場合、手動で初期誤差を設定する必要があります
* これは出力変数のgrad属性を設定することで簡単に実現できます。
* functionモジュールにVariable変数を返す関数が多数実装されており、これらを使って逆伝播関数を自動で計算できる

In [11]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = x**2 - 2*x + 1
y.grad = np.ones((2, 3), dtype=np.float32)
y.backward()
x.grad

array([[  0.,   2.,   4.],
       [  6.,   8.,  10.]], dtype=float32)

## Links
* NNを書くために、パラメータ付き関数をつなげてパラメータの最適化をする
* この時、linkを使う
* Linksはパラメータを保存するオブジェクト
* Linearがよく使われる
* 一次結合を表現するlink ($f(x)=Wx+b$)
* ↓に３次元空間から２次元空間へ写像する線形関数の定義を書く

In [12]:
f = F.Linear(3,2)

* ほとんどの関数はミニバッチ入力しか受け入れません。
* ここで、入力配列の最初の次元はバッチの次元として考えます。
* 線形関数の場合、入力は(N, 3)のようにNはミニバッチのサイズでなければなりません

* linkのパラメータは属性として格納
* それぞれのパラメータはVariable型のインスタンスとして生成
* 線形関数の場合、2つのパラメータWとbが格納
* 標準では、行列Wはランダムで初期化され、ベクトルbはゼロで初期化

In [13]:
f.W.data

array([[-0.86246145,  0.21997322, -1.22502005],
       [ 0.38768795, -1.66020846, -1.01530361]], dtype=float32)

In [14]:
f.b.data

array([ 0.,  0.], dtype=float32)

In [15]:
## fは通常の関数として動作する
x = Variable(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32))
y = f(x)
y.data

array([[ -4.09757519,  -5.9786396 ],
       [ -9.70009995, -12.84211254]], dtype=float32)

* パラメータの勾配はbackward()メソッドにより計算
* 最初に新しく計算するときに勾配を初期化しなければなりません
* zerograd()メソッドを呼ぶことになります。

In [16]:
f.zerograds()

In [17]:
y.grad = np.ones((2, 2), dtype=np.float32)
y.backward()


In [18]:
f.W.grad

array([[ 5.,  7.,  9.],
       [ 5.,  7.,  9.]], dtype=float32)

In [19]:
f.b.grad

array([ 2.,  2.], dtype=float32)

## Write a model as a chain
* ほとんどのニューラルネットワーク構造は複数のlinkを含みます。
* 例えば多層パーセプトロンは複数の線形層で構成されます。
* Chainerでは複数のlinkのおまとめ管理を次のように書きます:

In [20]:
l1 = L.Linear(4, 3)
l2 = L.Linear(3, 2)
def my_forward(x):
    h = l1(x)
    return l2(h)


* 直接チェインを定義すると再利用が難しいので、クラスで定義する

In [21]:
class MyProc(object):
    def __init__(self):
        self.l1 = L.Linear(4, 3)
        self.l2 = L.Linear(3, 2)
    
    def forward(self, x):
        h = self.l1(x)
        return self.l2(h)


* より再利用できるように、パラメータの管理、CPU/GPUマイグレーション、堅牢かつ柔軟なセーブ/ロード機能(v1.5以降)などをサポートしてるChainerのChainクラスから継承します。
* そのときの定義はこう書きます:

In [22]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            l1=L.Linear(4, 3),
            l2=L.Linear(3, 2),
        )
    def __call__(self, x):
        h = self.l1(x)
        return self.l2(h)
    

* l1やl2のようなlinksはMyChainの子linksと呼ばれます。Chain自体はLinkを継承しています。つまり子linksとしてMyChainオブジェクトを保持するもっと複雑なchainsを定義できます。

* 他のchainの定義の方法はChainListクラスを使うことです。このようにlinksのリストのように振る舞います。

In [23]:
class MyChain2(ChainList):
    def __init__(self):
        super(MyChain2, self).__init__(
            L.Linear(4, 3),
            L.Linear(3, 2),
        )
    
    def __call__(self, x):
        h = self[0](x)
        return self[1](h)
    

## Optimizer
* パラメーターが良い値になるように、Optimizerクラスで最適化を図ります。
* これは与えられたlinkの数値最適化アルゴリズムを実行します。
* たくさんのアルゴリズムがoptimizersモジュールに実装されています(SGDとかAdamとか)。
* ここではSGD(確率的勾配降下法)と呼ばれる最もシンプルなアルゴリズムのうちのひとつを見て見る

In [24]:
model = MyChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

* 最適化実行には二つの方法があります。
* ひとつはupdate()メソッド(引数なし)で呼び出し手動で勾配計算をすることです。 予め勾配はリセットすることを忘れないでください！(勾配は加算されてしまう)
* もう一つは、Loss Functionを与える

In [25]:
## updateメソッドを呼び出す方法
model.zerograds()
# compute gradient here...
optimizer.update()


## Serializer
* Serializeとは、他で扱えるようなデータ変換
* sirializersモジュールでそれらが定義されいます。今のところ、HDF5フォーマットです。
* linkオブジェクトをHDF5ファイルに変換するにはserializers.save_hdf5()を使います

* modelのパラメータをファイル名'my.model'でHDF5ファイルとして保存します。
* 保存されたmodelはserializers.load_hdf5()で読みこめます

In [26]:
serializers.save_hdf5('my.model', model)

In [27]:
serializers.load_hdf5('my.model', model)